In [ ]:
#pms = PyMinkSum
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import time
import Polytopes as pms
import setbasedmethods.convexsets._convexsets as pms2
reload(pms)

N = 1000
num_incorrect = 0
py_times = np.zeros(N)
pms_times = np.zeros(N)

overall_start_time = time.time()
for k in range(N):
# k=0
# while num_incorrect == 0:
#     k +=1

    #Random generator matrix
    #m = 3,4,5,6
    m = 3
    #n = 2,3,4,5
    n = 2
    gens = np.random.uniform(-1, 1, (n, m))
    iterates = product([-1, 1], repeat=m)
    X = np.empty((m, 1))
    for row in iterates: X = np.hstack((X, np.array(row).reshape(m, 1)))
    X = X[:,1:]
    
    #Polytopes representing the line segments
    py_start_time = time.time()
    poly_list = []
    for i in range(m):
        poly_list.append(pms.Polytope(np.hstack((-1*gens[:,i][:,None], 1*gens[:,i][:,None])).T))
    polytope_list=poly_list
    
    #This is the zonotope: the minkowski sum of the 3 line segments in R^2
    NV = pms.MinkSum(poly_list)
    py_end_time = time.time()
    NV = np.around(NV, 7)
    py_times[k] = py_end_time - py_start_time
    
    
    pms_start_time = time.time()
    poly_list2 = []
    for i in range(m):
        poly_list2.append(pms2.Polytope(np.hstack((-1*gens[:,i][:,None], 1*gens[:,i][:,None])).T))
    polytope_list2=poly_list2
    
    #This is the zonotope: the minkowski sum of the 3 line segments in R^2
    NV2 = pms2.Minksum(tuple(poly_list2))
    NV2 = NV2.generators()
    pms_end_time = time.time()
    NV2 = np.around(NV2, 7)
    pms_times[k] = pms_end_time - pms_start_time
    
    if NV.shape != NV2.shape:
        num_incorrect +=1
        continue
    for j in range(NV.shape[0]):
        if list(NV[j]) not in [list(NV2[l]) for l in range(NV2.shape[0])] or \
        list(NV2[j]) not in [list(NV[l]) for l in range(NV.shape[0])]:
            num_incorrect +=1
            break
overall_end_time = time.time()
print overall_end_time - overall_start_time
print (np.mean(py_times), np.mean(pms_times))
print 1.0*num_incorrect/N